###  Importing Library

In [36]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.utils import make_grid, save_image
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler


# 1. Hyperparameters & Device

In [37]:
batch_size = 128
image_size = 128       # resolusi gambar (64x64)
nz = 128              # dimensi vektor noise (input generator)
num_epochs = 300      # ganti sesuai kebutuhan
lr = 0.0005
beta1 = 0.5
ngf = 64  # generator feature map size
ndf = 64  # discriminator feature map size

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
    print("CUDA Version:", torch.version.cuda)

CUDA Available: True
GPU Name: NVIDIA GeForce GTX 1060 6GB
CUDA Version: 11.8


# 2. Dataset & DataLoader (dengan Augmentations)

In [38]:
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

In [39]:
data_dir = r"C:\Users\dawwi\Downloads\Dataset_3"  # sesuaikan path
dataset = torchvision.datasets.ImageFolder(root=data_dir, transform=transform)

indices = list(range(len(dataset)))
np.random.shuffle(indices)

# Misal pakai subset 1000 data
subset_indices = indices[:1000]
sampler = SubsetRandomSampler(subset_indices)

# DataLoader (menggunakan sampler)
dataloader = DataLoader(
    dataset,
    batch_size=batch_size,
    sampler=sampler,
    num_workers=4
)

In [40]:
# data_dir = r"C:\Users\dawwi\Downloads\Dataset"  # ganti dengan path dataset Anda
#
# transform = transforms.Compose([
#     transforms.Resize(image_size),
#     transforms.CenterCrop(image_size),
#     transforms.RandomRotation(degrees=15),
#     transforms.RandomHorizontalFlip(p=0.5),
#     transforms.ColorJitter(
#         brightness=(0.2, 0.6),   # contoh range brightness
#         contrast=(0.2, 0.8),
#         saturation=(0.2, 0.7),
#         hue=(-0.1, 0.1)
#     ),
#     transforms.ToTensor(),
#     transforms.Normalize([0.5]*3, [0.5]*3)
# ])
#
#
# dataset = torchvision.datasets.ImageFolder(root=data_dir, transform=transform)
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)

# 3. Definisi Model: Generator & Discriminator

In [41]:
class Generator(nn.Module):
    def __init__(self, nz, ngf, nc=3):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf*8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf*8),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf*8, ngf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*4),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*2),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.main(x)

In [42]:
class Discriminator(nn.Module):
    def __init__(self, ndf, nc=3):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf, ndf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf*2, ndf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*4),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf*4, ndf*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf*8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.main(x)

netG = Generator(nz, ngf).to(device)
netD = Discriminator(ndf).to(device)

# 4. Loss, Optimizer

In [43]:
criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

lossD_history = []
lossG_history = []

# noise tetap (untuk sample gambar di tiap epoch)
fixed_noise = torch.randn(batch_size, nz, 1, 1, device=device)

print("Start Training...")
num_iter = 0

real_label = 1.0
fake_label = 0.0

os.makedirs("samples", exist_ok=True)

Start Training...


# 5. Training Loop

In [44]:
for epoch in range(num_epochs):
    for i, (imgs, _) in enumerate(dataloader):
        # (A) Train Discriminator
        netD.zero_grad()

        real_imgs = imgs.to(device)
        b_size = real_imgs.size(0)

        label_real = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        label_fake = torch.full((b_size,), fake_label, dtype=torch.float, device=device)

        output_real = netD(real_imgs).view(-1)
        lossD_real = criterion(output_real, label_real)

        noise = torch.randn(b_size, nz, 1, 1, device=device)
        fake_imgs = netG(noise)
        output_fake = netD(fake_imgs.detach()).view(-1)
        lossD_fake = criterion(output_fake, label_fake)

        lossD_total = lossD_real + lossD_fake
        lossD_total.backward()
        optimizerD.step()

        # (B) Train Generator
        netG.zero_grad()
        # Generator wants D(G(z)) == 1
        output_fake_for_G = netD(fake_imgs).view(-1)
        lossG_total = criterion(output_fake_for_G, label_real)
        lossG_total.backward()
        optimizerG.step()

        lossD_history.append(lossD_total.item())
        lossG_history.append(lossG_total.item())

        num_iter += 1
        if i % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}] | Batch [{i}/{len(dataloader)}] | "
                  f"LossD: {lossD_total.item():.4f} | LossG: {lossG_total.item():.4f}")

    # Simpan sample di akhir epoch
    with torch.no_grad():
        fake = netG(fixed_noise).detach().cpu()
    grid = make_grid(fake, nrow=8, normalize=True)
    save_image(grid, f"samples/epoch_{epoch+1}.png")
    print(f"Sample image saved: samples/epoch_{epoch+1}.png")

print("Training Finished.")

ValueError: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([3200])) is deprecated. Please ensure they have the same size.

# 6. Plot Line Chart (Loss vs. Iteration)

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Training Loss (Discriminator & Generator)")
plt.plot(lossD_history, label="LossD")
plt.plot(lossG_history, label="LossG")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.legend()
plt.savefig("training_loss_plot.png")
plt.show()
print("Loss plot saved: training_loss_plot.png")

# 7. Visualisasi t-SNE di Latent Noise

In [ ]:
def visualize_latent_tsne(modelG, num_samples=500):
    modelG.eval()
    all_z = []
    for _ in range(num_samples):
        z = torch.randn(1, nz, 1, 1, device=device)
        # flatten
        all_z.append(z.view(1, -1).cpu().numpy())
    all_z = np.concatenate(all_z, axis=0)  # shape (num_samples, nz)

    print("Running t-SNE on latent vectors...")
    tsne = TSNE(n_components=2, perplexity=30, n_iter=1000)
    z_2d = tsne.fit_transform(all_z)

    plt.figure(figsize=(8,6))
    plt.scatter(z_2d[:,0], z_2d[:,1], alpha=0.7, s=10, c='blue')
    plt.title("t-SNE of Random Noise Z (DCGAN)")
    plt.xlabel("Dimension 1")
    plt.ylabel("Dimension 2")
    plt.show()
    print("t-SNE plot displayed.")

# 8. Eval Function (Contoh)

In [ ]:
def evaluate_model(generator, device, num_images=16, save_path="eval_samples.png"):
    """
    Contoh evaluasi sederhana: generate sejumlah gambar fake,
    lalu simpan dalam grid.
    """
    generator.eval()
    with torch.no_grad():
        noise = torch.randn(num_images, nz, 1, 1, device=device)
        fake_images = generator(noise).cpu()
    grid = make_grid(fake_images, nrow=4, normalize=True)
    save_image(grid, save_path)
    print(f"Evaluation samples saved: {save_path}")